In [7]:
from safetensors.torch import load_file
import pandas as pd
import numpy as np

state_dict = load_file("weights/model.safetensors")

In [22]:
df = {
    "key": [],
    "shape": []
}
for key, values in state_dict.items():
    df["key"].append(key)
    df["shape"].append(values.shape)
df = pd.DataFrame(df)
split = pd.Series(state_dict.keys()).str.split(".", expand=True)
df = pd.concat([df, split], axis=1)
df

,key,shape,0,1,2,3,4,5
0,backbone.blocks.0.filter.D,"(128,)",backbone,blocks,0,filter,D,None
1,backbone.blocks.0.filter.poles,"(128, 8, 1, 2)",backbone,blocks,0,filter,poles,None
2,backbone.blocks.0.filter.residues,"(128, 8, 1, 2)",backbone,blocks,0,filter,residues,None
3,backbone.blocks.0.filter.short_filter_bias,"(384,)",backbone,blocks,0,filter,short_filter_bias,None
4,backbone.blocks.0.filter.short_filter_weight,"(384, 1, 3)",backbone,blocks,0,filter,short_filter_weight,None
...,...,...,...,...,...,...,...,...
209,backbone.blocks.9.pre_norm.scale,"(128,)",backbone,blocks,9,pre_norm,scale,None
210,backbone.blocks.9.projections.bias,"(384,)",backbone,blocks,9,projections,bias,None
211,backbone.blocks.9.projections.weight,"(384, 128)",backbone,blocks,9,projections,weight,None
212,backbone.embedding_layer.weight,"(32, 128)",backbone,embedding_layer,weight,None,None,None


In [40]:
df[1].value_counts()

blocks             212
embedding_layer      1
norm                 1
Name: 1, dtype: int64

In [28]:
384/128

3.0

In [26]:
df.loc[df[2]=="0"]

,key,shape,0,1,2,3,4,5
0,backbone.blocks.0.filter.D,"(128,)",backbone,blocks,0,filter,D,None
1,backbone.blocks.0.filter.poles,"(128, 8, 1, 2)",backbone,blocks,0,filter,poles,None
2,backbone.blocks.0.filter.residues,"(128, 8, 1, 2)",backbone,blocks,0,filter,residues,None
3,backbone.blocks.0.filter.short_filter_bias,"(384,)",backbone,blocks,0,filter,short_filter_bias,None
4,backbone.blocks.0.filter.short_filter_weight,"(384, 1, 3)",backbone,blocks,0,filter,short_filter_weight,None
5,backbone.blocks.0.mlp.l1.weight,"(352, 128)",backbone,blocks,0,mlp,l1,weight
6,backbone.blocks.0.mlp.l2.weight,"(352, 128)",backbone,blocks,0,mlp,l2,weight
7,backbone.blocks.0.mlp.l3.weight,"(128, 352)",backbone,blocks,0,mlp,l3,weight
8,backbone.blocks.0.out_filter_dense.bias,"(128,)",backbone,blocks,0,out_filter_dense,bias,None
9,backbone.blocks.0.out_filter_dense.weight,"(128, 128)",backbone,blocks,0,out_filter_dense,weight,None


In [23]:
df[1].value_counts()

blocks             212
embedding_layer      1
norm                 1
Name: 1, dtype: int64

In [24]:
np.sort(df.loc[df[1]=="blocks",2].unique().astype(int)).astype(str)

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15'], dtype='<U21')

There are 16 blocks.

The number of blocks is controlled by `config.num_layers`

In [25]:
layer_idxs = np.sort(df.loc[df[1]=="blocks",2].unique().astype(int)).astype(str)
for layer_idx in layer_idxs:
    idx = (df[1]=="blocks") & (df[2]==layer_idx)
    block = df.loc[idx]
    print(layer_idx, block[3].value_counts().index)

0 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
1 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
2 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
3 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
4 Index(['inner_mha_cls', 'mlp', 'post_norm', 'pre_norm'], dtype='object')
5 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
6 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
7 Index(['filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm'],
      dtype='object')
8 Index(['inner_mha_cls', 'mlp', 'post_norm', 'pre_norm'], dtype='object')
9 Index(['filter', 'mlp', 'out_filter_

- Hyena layers: 'filter', 'mlp', 'out_filter_dense', 'projections', 'post_norm',
       'pre_norm' --> [0,1,2,3,5,6,7,9,10,11,13,14,15]
- Attention layers: 'inner_mha_cls', 'mlp', 'post_norm', 'pre_norm' --> [4,8,12]

In [19]:
df

,0,1,2,3,4,5
0,backbone,blocks,0,filter,D,None
1,backbone,blocks,0,filter,poles,None
2,backbone,blocks,0,filter,residues,None
3,backbone,blocks,0,filter,short_filter_bias,None
4,backbone,blocks,0,filter,short_filter_weight,None
...,...,...,...,...,...,...
209,backbone,blocks,9,pre_norm,scale,None
210,backbone,blocks,9,projections,bias,None
211,backbone,blocks,9,projections,weight,None
212,backbone,embedding_layer,weight,None,None,None


## check

```
python training-model/get_likelihoods.py \
            --checkpoint="weights/model.safetensors" \
            --tokenizer="processing-seqs/lornash_tokenizer.json" \
            --dataset="tokenization" \
            --dataset_type="example" \
            --predictions_dir="predictions"
Traceback (most recent call last):
  File "training-model/get_likelihoods.py", line 88, in <module>
    model.load_state_dict(state_dict)
  File "/home/manglada/miniforge3/envs/lornash/lib/python3.8/site-packages/torch/nn/modules/module.py", line 2215, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for StripedHyenaModelForCausalLM:
	Missing key(s) in state_dict: "backbone.unembed.weight", "backbone.flash_fft.f_16_fft_real", "backbone.flash_fft.f_16_ifft_real", "backbone.flash_fft.f_16_fft_imag", "backbone.flash_fft.f_16_ifft_imag", "backbone.flash_fft.f_16_fft", "backbone.flash_fft.f_16_ifft", "backbone.flash_fft.twiddle_factors_fft_16_16", "backbone.flash_fft.twiddle_factors_ifft_16_16", "backbone.flash_fft.twiddle_factors_fft_16_256", "backbone.flash_fft.twiddle_factors_ifft_16_256", "backbone.flash_fft.twiddle_factors_fft_real", "backbone.flash_fft.twiddle_factors_ifft_real", "backbone.flash_fft.twiddle_factors_fft_imag", "backbone.flash_fft.twiddle_factors_ifft_imag", "backbone.blocks.0.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.0.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.0.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.0.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.0.filter.fftconv_fn.f_16_fft", "backbone.blocks.0.filter.fftconv_fn.f_16_ifft", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.0.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.1.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.1.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.1.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.1.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.1.filter.fftconv_fn.f_16_fft", "backbone.blocks.1.filter.fftconv_fn.f_16_ifft", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.1.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.2.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.2.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.2.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.2.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.2.filter.fftconv_fn.f_16_fft", "backbone.blocks.2.filter.fftconv_fn.f_16_ifft", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.2.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.3.inner_mha_cls.rotary_emb.inv_freq", "backbone.blocks.3.inner_mha_cls.Wqkv.weight", "backbone.blocks.3.inner_mha_cls.out_proj.weight", "backbone.blocks.4.filter.D", "backbone.blocks.4.filter.short_filter_weight", "backbone.blocks.4.filter.short_filter_bias", "backbone.blocks.4.filter.poles", "backbone.blocks.4.filter.residues", "backbone.blocks.4.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.4.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.4.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.4.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.4.filter.fftconv_fn.f_16_fft", "backbone.blocks.4.filter.fftconv_fn.f_16_ifft", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.4.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.4.projections.weight", "backbone.blocks.4.projections.bias", "backbone.blocks.4.out_filter_dense.weight", "backbone.blocks.4.out_filter_dense.bias", "backbone.blocks.5.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.5.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.5.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.5.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.5.filter.fftconv_fn.f_16_fft", "backbone.blocks.5.filter.fftconv_fn.f_16_ifft", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.5.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.6.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.6.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.6.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.6.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.6.filter.fftconv_fn.f_16_fft", "backbone.blocks.6.filter.fftconv_fn.f_16_ifft", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.6.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.7.inner_mha_cls.rotary_emb.inv_freq", "backbone.blocks.7.inner_mha_cls.Wqkv.weight", "backbone.blocks.7.inner_mha_cls.out_proj.weight", "backbone.blocks.8.filter.D", "backbone.blocks.8.filter.short_filter_weight", "backbone.blocks.8.filter.short_filter_bias", "backbone.blocks.8.filter.poles", "backbone.blocks.8.filter.residues", "backbone.blocks.8.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.8.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.8.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.8.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.8.filter.fftconv_fn.f_16_fft", "backbone.blocks.8.filter.fftconv_fn.f_16_ifft", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.8.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.8.projections.weight", "backbone.blocks.8.projections.bias", "backbone.blocks.8.out_filter_dense.weight", "backbone.blocks.8.out_filter_dense.bias", "backbone.blocks.9.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.9.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.9.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.9.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.9.filter.fftconv_fn.f_16_fft", "backbone.blocks.9.filter.fftconv_fn.f_16_ifft", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.9.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.10.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.10.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.10.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.10.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.10.filter.fftconv_fn.f_16_fft", "backbone.blocks.10.filter.fftconv_fn.f_16_ifft", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.10.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.11.inner_mha_cls.rotary_emb.inv_freq", "backbone.blocks.11.inner_mha_cls.Wqkv.weight", "backbone.blocks.11.inner_mha_cls.out_proj.weight", "backbone.blocks.12.filter.D", "backbone.blocks.12.filter.short_filter_weight", "backbone.blocks.12.filter.short_filter_bias", "backbone.blocks.12.filter.poles", "backbone.blocks.12.filter.residues", "backbone.blocks.12.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.12.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.12.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.12.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.12.filter.fftconv_fn.f_16_fft", "backbone.blocks.12.filter.fftconv_fn.f_16_ifft", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.12.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.12.projections.weight", "backbone.blocks.12.projections.bias", "backbone.blocks.12.out_filter_dense.weight", "backbone.blocks.12.out_filter_dense.bias", "backbone.blocks.13.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.13.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.13.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.13.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.13.filter.fftconv_fn.f_16_fft", "backbone.blocks.13.filter.fftconv_fn.f_16_ifft", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.13.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.14.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.14.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.14.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.14.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.14.filter.fftconv_fn.f_16_fft", "backbone.blocks.14.filter.fftconv_fn.f_16_ifft", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.14.filter.fftconv_fn.twiddle_factors_ifft_imag", "backbone.blocks.15.filter.fftconv_fn.f_16_fft_real", "backbone.blocks.15.filter.fftconv_fn.f_16_ifft_real", "backbone.blocks.15.filter.fftconv_fn.f_16_fft_imag", "backbone.blocks.15.filter.fftconv_fn.f_16_ifft_imag", "backbone.blocks.15.filter.fftconv_fn.f_16_fft", "backbone.blocks.15.filter.fftconv_fn.f_16_ifft", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_fft_16_16", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_ifft_16_16", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_fft_16_256", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_ifft_16_256", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_fft_real", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_ifft_real", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_fft_imag", "backbone.blocks.15.filter.fftconv_fn.twiddle_factors_ifft_imag". 
	Unexpected key(s) in state_dict: "backbone.blocks.3.filter.D", "backbone.blocks.3.filter.poles", "backbone.blocks.3.filter.residues", "backbone.blocks.3.filter.short_filter_bias", "backbone.blocks.3.filter.short_filter_weight", "backbone.blocks.3.out_filter_dense.bias", "backbone.blocks.3.out_filter_dense.weight", "backbone.blocks.3.projections.bias", "backbone.blocks.3.projections.weight", "backbone.blocks.4.inner_mha_cls.Wqkv.bias", "backbone.blocks.4.inner_mha_cls.Wqkv.weight", "backbone.blocks.4.inner_mha_cls.out_proj.bias", "backbone.blocks.4.inner_mha_cls.out_proj.weight", "backbone.blocks.4.inner_mha_cls.rotary_emb.inv_freq", "backbone.blocks.7.filter.D", "backbone.blocks.7.filter.poles", "backbone.blocks.7.filter.residues", "backbone.blocks.7.filter.short_filter_bias", "backbone.blocks.7.filter.short_filter_weight", "backbone.blocks.7.out_filter_dense.bias", "backbone.blocks.7.out_filter_dense.weight", "backbone.blocks.7.projections.bias", "backbone.blocks.7.projections.weight", "backbone.blocks.8.inner_mha_cls.Wqkv.bias", "backbone.blocks.8.inner_mha_cls.Wqkv.weight", "backbone.blocks.8.inner_mha_cls.out_proj.bias", "backbone.blocks.8.inner_mha_cls.out_proj.weight", "backbone.blocks.8.inner_mha_cls.rotary_emb.inv_freq", "backbone.blocks.11.filter.D", "backbone.blocks.11.filter.poles", "backbone.blocks.11.filter.residues", "backbone.blocks.11.filter.short_filter_bias", "backbone.blocks.11.filter.short_filter_weight", "backbone.blocks.11.out_filter_dense.bias", "backbone.blocks.11.out_filter_dense.weight", "backbone.blocks.11.projections.bias", "backbone.blocks.11.projections.weight", "backbone.blocks.12.inner_mha_cls.Wqkv.bias", "backbone.blocks.12.inner_mha_cls.Wqkv.weight", "backbone.blocks.12.inner_mha_cls.out_proj.bias", "backbone.blocks.12.inner_mha_cls.out_proj.weight", "backbone.blocks.12.inner_mha_cls.rotary_emb.inv_freq". 
    
```

- mha_out_proj_bias: True
- qkv_proj_bias: True

In [38]:
128 // 16 // 2

8

- num_attention_heads: 16

In [18]:
from configuration_hyena import StripedHyenaConfig
from modeling_hyena import StripedHyenaModelForCausalLM

config_dict = {
    "vocab_size": 32,                   # Effective vocabulary size (from RNA tokens in the paper)
    "hidden_size": 128,                 # Model width per block (from the architecture details)
    "num_filters": 128,                 # Filters align with model width
    "inner_mlp_size": 352,             # Approximation; might require refinement from tensor inspection
    "attn_layer_idxs": [4,8,12],      # Interleaved attention layers (as described in the architecture)
    "hyena_layer_idxs": [0,1,2,3,5,6,7,9,10,11,13,14,15], # Remaining layers are Hyena layers
    "num_layers": 16,                   # Total number of layers
    "tie_embeddings": False,            # No mention of tied embeddings
    "short_filter_length": 3,           # Typical short filter length
    "num_attention_heads": 4,           # Guess for attention heads; adjust as necessary
    "proj_groups": 1,                   # Default grouping for projections
    "hyena_filter_groups": 1,           # Based on typical Hyena usage
    "split_k0": True,                   # Split kernel optimization
    "column_split_hyena": True,         # Optimize for column splits
    "column_split": False,              # Default unless specified
    "model_parallel_size": 1,           # Single GPU (adjust based on hardware)
    "pipe_parallel_size": 1,            # Single pipeline (adjust if multi-pipeline is known)
    "short_filter_bias": True,          # Default
    "mha_out_proj_bias": False,         # Default for Hyena
    "qkv_proj_bias": False,             # Default
    "final_norm": True,                 # Normalize at the end
    "use_cache": True,                  # Default
    "use_flash_attention_2": True,      # Optimized attention mechanism
    "use_flash_rmsnorm": True,          # Efficient normalization
    "use_flash_depthwise": False,       # Default unless explicitly mentioned
    "use_flashfft": True,               # Hyena leverages FFT for speed
    "inference_mode": False,            # Default for training
    "prefill_style": "fft",             # FFT-based prefill (specific to Hyena-style computation)
    "max_seqlen": 65536,                # Maximum context length (65k as stated in the paper)
    "eps": 1e-5,                        # Default epsilon for numerical stability
    "state_size": 8,                    # Default
    "rotary_emb_base": 500000,          # Rotary embeddings
    "smeared_gqa": False,               # Default unless specified
    "make_vocab_size_divisible_by": 8,  # Common optimization for efficient computation
    "log_intermediate_values": False,   # Default
    "bidirectional": False              # Causal model
}
config_dict["seqlen"] = config_dict["max_seqlen"]

config = StripedHyenaConfig(**config_dict)
model = StripedHyenaModelForCausalLM(config)
model.load_state_dict(state_dict)

ModuleNotFoundError: No module named 'configuration_hyena'

```
python training-model/get_likelihoods.py             --checkpoint="weights/model.safetensors"             --tokenizer="processing-seqs/lornash_tokenizer.json"             --dataset="tokenization"             --dataset_type="example"             --predictions_dir="predictions"
model state_dict: ['backbone.embedding_layer.weight', 'backbone.norm.scale', 'backbone.unembed.weight']
model.safetensors: ['backbone.embedding_layer.weight', 'backbone.norm.scale']
Traceback (most recent call last):
  File "training-model/get_likelihoods.py", line 91, in <module>
    model.load_state_dict(state_dict)
  File "/home/manglada/miniforge3/envs/lornash/lib/python3.8/site-packages/torch/nn/modules/module.py", line 2215, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for StripedHyenaModelForCausalLM:
	Missing key(s) in state_dict: "backbone.unembed.weight". 
```

In [42]:
1

1

In [43]:
1

1

In [44]:
1

1